
<span type="title">代理模式</span> | <span type="update">2018-09-10</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍代理模式。代理模式指的是，通过为另一个对象昂提供替身或者占位符的方式，控制对这个对象的访问。其一般通过和被代理对象实现共同的接口，调用内部保存的被代理对象实例返回这些接口提供的服务。服务请求方只和代理进行交互。这种模式广泛应用于需要保证安全、轻量的场合，比如金融和网络。典型的代理模式是RMI，本地的客户端通过调用Stub类来访问远程服务，这个Stub类就是远程对象的代理。</p></span>

# 代理模式

## 代理模式的定义

代理模式指的是，通过为另一个对象提供替身或者占位符的方式，控制对这个对象的访问。

代理模式的实现常常采用包含一个被代理对象的引用，然后调用此对象和其共同实现的接口提供有限的服务。

![](proxy.png)

## RMI 概述

RMI 通过以下过程调用远程访问：

![](proxy1.png)
![](proxy2.png)
![](proxy3.png)

在最开始，本地对象通过Naming或者LocateRegistry通过Socket从远程服务器地址查找Stub对象，然后获取这个对象。

可以看到，其中的Skeleton负责监控端口的请求，以将远程对象通过序列化的方式发送到本地的Stub代理处。本地对象通过访问Stub即可获取远程对象所实现的之前约定好的接口的方法。

当调用Stub的方法的时候，Stub会联系远程对象请求方法，然后会送给客户端。这样，所有对于远程方法的请求都需要经过Stub到Skeleton到远程对象。

# RMI 示例代码

## 定义接口

首先要协商好传送的对象和服务端能够提供的方法，需要继承自 Remote 类，每个方法需要抛出 RemoteException 异常。

```java
public interface AutoMatRemote extends Remote {
    State getState() throws RemoteException;
    int getCount() throws RemoteException;
    String getLocation() throws RemoteException;
}
```

## 服务端

首先，客户端需要和服务端协商好能够调用的方法，Stub只能提供这些方法的调用。从远程对象获取的序列化流只能转型成为此接口的类型。此外，对于所有需要服务端进行调用时传送的类进行序列化声明，比如State类。之后，对于服务端实现 UnicastRemoteObject 类的继承以直接绑定和传送。这个 URO 类提供了很多用于二进制化和传送的方法。如果不继承，那么需要手动序列化再绑定到注册表。

此外，这个服务端的构造器需要能够抛出 RemoteException 异常。

直接运行即可，可以创建多个对象，绑定到相同接口，使用不同的名称来区分。可以使用LocateRegistry注册本地接口，使用bind方法进行绑定。程序不会像正常那样运行完就终止，其会停留在那里，等待接受客户端访问。

```java
public interface State extends Serializable {
    void insertQuarter();
    void ejectQuarter();
    void turnCrank();
    void dispense();
}
public class AutoMat extends UnicastRemoteObject implements Serializable, AutoMatRemote {
    public static void main(String[] args) throws Exception {
        AutoMat machine = new AutoMat("Wuhan CCNU",3);
        AutoMat machine2 = new AutoMat("Wuhan HZAU",10);
        AutoMat machine3 = new AutoMat("Guangzhou SCNU",8);
        Registry registry = LocateRegistry.createRegistry(2333);
        registry.rebind("scnu",machine3);
        registry.rebind("hzau",machine2);
        registry.rebind("ccnu",machine);
        System.out.println("Hi");
        machine.insertQuarter();
        machine.ejectQuarter();
        machine.insertQuarter();
        machine.turnCrank();
        machine.turnCrank();
    }
}
```

## 客户端

客户端只用使用注册表来查找指定网址的端口的字段来获得对象，其实现在获得的是Stub代理，转型为接口方法，然后调用返回远程方法的内容即可。

```java
Registry registry = LocateRegistry.getRegistry(2333);
for (String locate : "ccnu scnu hzau".split(" ")) {
    AutoMatRemote autoMat = (AutoMatRemote)registry.lookup(locate);
    System.out.printf("Machine Location: %s\nState: %s\nLeft Ball:%s\n",
            autoMat.getLocation(),autoMat.getState().toString(),autoMat.getCount());
}
//Result
Machine Location: Wuhan CCNU
State: noQuarterState@238e0d81
Left Ball:2
Machine Location: Guangzhou SCNU
State: noQuarterState@533ddba
Left Ball:8
Machine Location: Wuhan HZAU
State: noQuarterState@306a30c7
Left Ball:10
```